In [1]:
from dataclasses import dataclass, field
import open3d as o3d
import numpy as np
import time
from pathlib import Path
from bbox import  BBox3D
from scipy.spatial.transform import Rotation as R

# # The detected car is being overtaken.
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/000/000003/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/000/000003.npy",  allow_pickle=True).item()
# track_uuid = '293fe4b1-c557-4d2f-a214-9308a50f153b'

# A car is driving in the opposite direction and is turning. -> weird
pcd_track_uuids = np.load("../../data/P04/cleaned_data/000/000009/pcd.npy",  allow_pickle=True).item()
gt = np.load("../../data/P04/gt/000/000009.npy",  allow_pickle=True).item()
track_uuid = 'a02314d7-191c-496f-9b01-2dec0602e765'

# # The car drives in a perpendicular direction.
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/000/000011/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/000/000011.npy",  allow_pickle=True).item()
# track_uuid = '073a72b3-1750-4897-9c2c-0944a0d4ac89'

# A car is turning right. -> weird
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/000/000021/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/000/000021.npy",  allow_pickle=True).item()
# track_uuid = 'ffdcdedb-067d-479c-92c4-72c4ba7c993a'

# The car drives in a perpendicular direction.
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/000/000030/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/000/000030.npy",  allow_pickle=True).item()
# track_uuid = '2383ced0-16f8-4732-aede-142a8f4c6b04'

# # The ego-car is being overtaken
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/000/000049/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/000/000049.npy",  allow_pickle=True).item()
# track_uuid = '2d1e69f6-9aa9-4b89-a700-c51d12ec112c'

# The ego-car is turning left  -> weird
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/001/001000/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/001/001000.npy",  allow_pickle=True).item()
# track_uuid = '66f6a5e9-c37a-45e6-b3f8-00013daccb56'

# # The detected car is turning right and then goes straight
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/001/001005/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/001/001005.npy",  allow_pickle=True).item()
# track_uuid = 'be464e86-89ee-40d7-a417-2de73cac0a42'

# # The detected car comes from an alley and merges into the main road.
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/001/001006/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/001/001006.npy",  allow_pickle=True).item()
# track_uuid = '428c8029-4f08-4182-ae5e-92581d303779'

# # The detected car goes straight and turns right.
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/001/001007/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/001/001007.npy",  allow_pickle=True).item()
# track_uuid = '051af1a0-3b0c-41bc-88e7-10ada6eb10cf'

# The detected car comes from the opposite direction and turns left.
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/001/001027/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/001/001027.npy",  allow_pickle=True).item()
# track_uuid = 'b0dd7489-5721-4202-a700-3364adf5e461'

# The detected car is turning right
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/001/001035/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/001/001035.npy",  allow_pickle=True).item()
# track_uuid = '5ed56a87-e8c9-4e06-bc9a-ceb4a8a44360'

# # The detected car is turning left
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/001/001038/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/001/001038.npy",  allow_pickle=True).item()
# track_uuid = '1d33dc72-e987-4140-9a7f-b653b2d3b41d'

# # The car drives in perpendicular direction
# pcd_track_uuids = np.load("../../data/P04/cleaned_data/001/001039/pcd.npy",  allow_pickle=True).item()
# gt = np.load("../../data/P04/gt/001/001039.npy",  allow_pickle=True).item()
# track_uuid = 'ff60da28-a51a-4fdc-aceb-fd9605addaa0'

# # the car u-turns - hard to clean #shashank
# yaw_filter=[[0, 180], [-0, -50]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001046/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['945feec5-d2f5-49da-ab6d-c71f9402a23f']

instance = pcd_track_uuids[track_uuid]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
@dataclass
class BoundingBox3D:
    '''
    pose in s-frame
    '''
    x: float
    y: float
    z: float
    length: float
    width: float
    height: float
    rot: float
    iou: BBox3D = field(init=False, repr=False)

    def __post_init__(self):
        r = R.from_matrix(self.rot)
        q8d_xyzw = r.as_quat()
        q8d = np.array([q8d_xyzw[3], q8d_xyzw[0], q8d_xyzw[1], q8d_xyzw[2]])

        self.iou: BBox3D = BBox3D(self.x, self.y, self.z, 
                                         self.length, self.width, 
                                         self.height, q=q8d)

In [3]:
def translate_boxes_to_open3d_instance(bbox, crop=False):
    """
          4 -------- 6
         /|         /|
        5 -------- 3 .
        | |        | |
        . 7 -------- 1
        |/         |/
        2 -------- 0
    https://github.com/open-mmlab/OpenPCDet/blob/master/tools/visual_utils/open3d_vis_utils.py
    """
    center = [bbox.x, bbox.y, bbox.z]
    lwh = [bbox.length, bbox.width, bbox.height]
    if not crop:
        box3d = o3d.geometry.OrientedBoundingBox(center, bbox.rot, lwh)
    else:
        lwh = [bbox.length, bbox.width, bbox.height * 0.9]
        box3d = o3d.geometry.OrientedBoundingBox(center, bbox.rot, lwh)

    line_set = o3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = o3d.utility.Vector2iVector(lines)
    

    return line_set, box3d

In [4]:
def change_bbox(line_set, bbox):
    center = [bbox.x, bbox.y, bbox.z]
    lwh = [bbox.length, bbox.width, bbox.height]
    box3d = o3d.geometry.OrientedBoundingBox(center, bbox.rot, lwh)
        
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = o3d.utility.Vector2iVector(lines)

In [5]:
def get_bbox(pcd):
    x = pcd['bbox'][0]
    y = pcd['bbox'][1]
    z = pcd['bbox'][2]
    l = pcd['bbox'][4]
    w = pcd['bbox'][3]
    h = pcd['bbox'][5]
    rot = pcd['T_cam_obj'][:3, :3]
    bbox = BoundingBox3D(x,y,z,l,w,h,rot)
    return bbox

In [6]:
def clean_data(mtx, yaw_filter=[[0, 180], [-0, -180]]):
    r = R.from_matrix(mtx[:3, :3])
    euler = r.as_euler('zxy', degrees=True)[0]
    print("euler", euler)
    if euler < 0:
        if euler > yaw_filter[1][0] or euler < yaw_filter[1][1]:
            print("before minus", euler)
            mtx[0][0] = -mtx[0][0] 
            mtx[0][1] = -mtx[0][1] 
            mtx[1][0] = -mtx[1][0] 
            mtx[1][1] = -mtx[1][1]
            print("after minus", euler)
            return mtx
    else:
        if euler < yaw_filter[0][0] or euler > yaw_filter[0][1]:
            print("before plus", euler)
            mtx[0][0] = -mtx[0][0] 
            mtx[0][1] = -mtx[0][1] 
            mtx[1][0] = -mtx[1][0] 
            mtx[1][1] = -mtx[1][1]
            print("after plus", euler)
            return mtx
    return mtx

In [7]:
print(gt)

{0: {'1559ddf3-00d7-4ed7-8c23-7101b1b2f84f': {'x': 9.799027662274284, 'y': 27.31163324739549, 'z': -0.6729441443068627, 'length': 4.174820899963379, 'width': 1.74, 'height': 1.4356021881103516, 'rot': array([[-0.7017195 ,  0.71245333,  0.        ],
       [-0.71245333, -0.7017195 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])}, 'e5d92343-9257-47f0-a54f-029bafd1c83e': {'x': 38.36893095362643, 'y': -2.9456550418013308, 'z': 0.4659279586643379, 'length': 4.666590213775635, 'width': 1.8401176929473877, 'height': 1.8508014678955078, 'rot': array([[ 0.99993055, -0.01178542,  0.        ],
       [ 0.01178542,  0.99993055,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])}, 'f9ab9e33-4c5c-45e9-968e-c105b32012c8': {'x': 15.630318060126683, 'y': 32.78128983986272, 'z': -0.8741512925978316, 'length': 4.03, 'width': 1.74, 'height': 1.4698171615600586, 'rot': array([[-0.71691404,  0.69716157,  0.        ],
       [-0.69716157, -0.71691404,  0.        ],
       [ 0.   

In [8]:
# seq = Path("001/001038")

# result_dir = Path("/home/ohmpr/master_bonn/Modules/3rd_semester/P02/sandbox/Deep_SDF_Debug/data/P04_pcd") / seq

# if not result_dir.exists():
#     print("test")
#     result_dir.mkdir(parents=True, exist_ok=True)

instance_id = 0
max_frame = 0
for k, _ in instance.items():
    frame_number = len(instance[k])
    if frame_number > max_frame:
        max_frame = frame_number
        first_instance = k
points = instance[first_instance]
# Find the first frame
first_frame = 0
for k, v in points.items():
    first_frame = k
    break

# first_frame = 78
# Visualize results
vis = o3d.visualization.Visualizer()
vis.create_window()
# Coordinate frame
axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
vis.add_geometry(axis_pcd)

pts_str = 'pts_cam' # dict_keys(['T_cam_obj', 'pts_cam', 'surface_points', 'bbox'])


############# Bounding boxes #############
# Check wrong yaw detection
mtx = points[first_frame]['T_cam_obj']
# Extracted point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[first_frame][pts_str])
vis.add_geometry(pcd)
# o3d.io.write_point_cloud(f"{result_dir}/{first_frame}.pcd", pcd)

# Orientation
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3, origin=[0, 0, 0])
coordinate_frame.transform(mtx)
prev_mtx = mtx
vis.add_geometry(coordinate_frame)

# Detected bounding box 
bbox = get_bbox(points[first_frame])
line_set, box3d = translate_boxes_to_open3d_instance(bbox)
line_set.paint_uniform_color(np.array([60. / 255., 180. / 255., 75. / 255.])) # GREEN
vis.add_geometry(line_set)

############# Bounding boxes #############


############# Ground truth #############
gt_bbox = get_bbox_gt(gt[first_frame][track_uuid])
gt_line_set, gt_box3d = translate_boxes_to_open3d_instance(gt_bbox)
gt_line_set.paint_uniform_color(np.array([230. / 255., 0., 0.]))  # red

gt_mtx = np.hstack((gt[first_frame][track_uuid]['rot'], np.array([[gt[first_frame][track_uuid]['x']], 
                                                                  [gt[first_frame][track_uuid]['y']], 
                                                                  [gt[first_frame][track_uuid]['z']]])))
gt_mtx = np.vstack((gt_mtx, np.array([0, 0, 0, 1])))
prev_gt_mtx = gt_mtx
vis.add_geometry(gt_line_set)
############# Ground truth #############

for frame_id, points_scan in points.items():
    if frame_id == first_frame:
        pass

    ############# Bounding boxes #############
    # Point Cloud
    pcd.points = o3d.utility.Vector3dVector(points_scan[pts_str])
    # o3d.io.write_point_cloud(f"{result_dir}/{frame_id}.pcd", pcd)
    vis.update_geometry(pcd)
    
    # Orientation
    coordinate_frame.transform(np.linalg.inv(prev_mtx)) # undo previous transformation
    mtx = points_scan['T_cam_obj']
    
    # print("frame_id", frame_id)
    coordinate_frame.transform(mtx)
    vis.update_geometry(coordinate_frame)
    
    # Detected bounding box 
    line_set.transform(np.linalg.inv(prev_mtx)) # undo previous transformation
    bbox = get_bbox(points_scan)
    change_bbox(line_set, bbox)
    line_set.transform(mtx)
    line_set.paint_uniform_color(np.array([60. / 255., 180. / 255., 75. / 255.])) # GREEN
    vis.update_geometry(line_set)
    prev_mtx = mtx
    ############# Bounding boxes #############


    ############# Ground truth #############
    gt_mtx = np.hstack((gt[frame_id][track_uuid]['rot'], np.array([[gt[frame_id][track_uuid]['x']], 
                                                                      [gt[frame_id][track_uuid]['y']], 
                                                                      [gt[frame_id][track_uuid]['z']]])))
    gt_mtx = np.vstack((gt_mtx, np.array([0, 0, 0, 1])))

    gt_line_set.transform(np.linalg.inv(prev_gt_mtx)) # undo previous transformation
    gt_bbox = get_bbox_gt(gt[frame_id][track_uuid])
    change_bbox(gt_line_set, gt_bbox)
    gt_line_set.transform(gt_mtx)
    gt_line_set.paint_uniform_color(np.array([230. / 255., 0., 0.]))  # red

    vis.update_geometry(gt_line_set)

    prev_gt_mtx = gt_mtx
    ############# Ground truth #############
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)

# vis.run()
vis.destroy_window()

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


NameError: name 'get_bbox_gt' is not defined